In [4]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score

### 1. importing train dataset

In [5]:
df = pd.read_csv('04 model test data.csv')
mc = [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1]
ic = ['race','gender','age','admission_type_id','admission_source_id','metformin','nateglinide',
      'glipizide','glyburide','pioglitazone','insulin','diabetesMed']

In [6]:
for j in ic:
    df[j] = df[j].astype('category')
y = df['readmitted']
x = df.drop(['readmitted'], axis=1)

### 2. training the model

In [7]:
model = lgb.LGBMClassifier(objective='binary',metric='auc',boosting_type='gbdt',
                           monotone_constraints=mc,categorical_feature=ic,missing='?',
                           lambda_l1=0.5,lambda_l2=1,learning_rate=0.1,max_depth=20,
                           min_data_in_leaf=50,n_estimators=50,random_state=0,verbose=-1)

### 3. pred

In [8]:
model.fit(x,y)
print(x.shape)
pred2 = []
pred1 = model.predict_proba(x)[:,1]
a1 = roc_auc_score(y,pred1)
print('auc roc  :',np.round(a1,3))

for j in pred1:
    if j > 0.30: pred2.append(1)
    else: pred2.append(0)
c1 = confusion_matrix(y,pred2)
p = c1[1][1] / (c1[0][1]+c1[1][1])
r = c1[1][1] / (c1[1][0]+c1[1][1])
f1 = (2*p*r) / (p+r)
print('f1 score :',np.round(f1,3))

(10000, 19)
auc roc  : 0.726
f1 score : 0.625
